In [1]:
from skimage import data
from scipy import ndimage as ndi
import napari
from skimage import measure
import pandas as pd
from tqdm import tqdm
import trimesh

# return volume for label from image
def label_volume_from_label_props(labels, label, props_table, spatial_axis, ):
    # create slices
    labels_slices = [slice(None) for x in range(len(labels.shape))]
    
    # get label props from table
    label_props = props_table[props_table['label'] == label].iloc[0]
    
    for i, x in spatial_axis.items():
        bbox_min = max(int(label_props[f'bbox-min-{i}']), 0)
        bbox_max = min(int(label_props[f'bbox-max-{i}']), labels.shape[x])
        
        labels_slices[x] = slice(bbox_min, bbox_max, 1)
    
    return labels[tuple(labels_slices)] == label

# generate mesh from labelled volume
def mesh_from_label_volume(volume, spacing = 1.0,
                           simplify = False, simplify_factor = 0.8,
                           process = True):
    # make 2D surface to 3D
    if len(volume.shape) == 2:
        volume = np.expand_dims(volume, axis = 0)
                               
    # create volume - used skimage.measure.marching_cubes internally
    # pitch == spacing
    volume_mesh = trimesh.voxel.ops.matrix_to_marching_cubes(
        volume, pitch = spacing
    )
    
    # make sure it is watertight if not
    # TODO could not find a better solution
    if volume_mesh.fill_holes() is False:
        volume_mesh = volume_mesh.split(only_watertight = True)[0]
    
    # does that help ..?
    volume_mesh.merge_vertices(merge_tex = True, merge_norm = True)
    
    # process mesh
    if process is True:
        volume_mesh.process()
    
    # simplify object
    if simplify is True:
        volume_mesh = volume_mesh.simplify_quadratic_decimation(
            len(volume_mesh.faces) * simplify_factor)
    
    return volume_mesh

# define axis and scale
spatial_axis = {'z':0, 'y':1, 'x':2}
im_scale = [10, 0.992, 0.992]

# open image and segmentation
blobs = data.binary_blobs(length=128, volume_fraction=0.1, n_dim=3)
viewer = napari.view_image(blobs.astype(float), name='blobs')
labeled = ndi.label(blobs)[0]
labels_layer = viewer.add_labels(labeled, name='blob ID')
viewer.dims.ndisplay = 3

props_to_get = [
    'label',
    'mean_intensity',
    'centroid',
    'bbox',
]

# measure props
props_table = pd.DataFrame(
    measure.regionprops_table(labeled, intensity_image=blobs, properties=props_to_get))

# rename bbox columns with _min_x, _max_x, ...
# get min
rename_cols = {
    'bbox-' + str(x): 'bbox-min-' + str(i).lower()
    for i, x in spatial_axis.items() if x is not None
}

# get max
rename_cols.update({
    'bbox-' + str(x + len(spatial_axis)): 'bbox-max-' + str(i).lower()
    for i, x in spatial_axis.items() if x is not None
})

# apply
props_table.rename(columns = rename_cols, inplace = True)

# save shapes
shape_descriptors = list()

# go through labels
for i in tqdm(props_table['label']):
    # get label volume
    volume = label_volume_from_label_props(labeled, i, props_table, spatial_axis = spatial_axis)

    # get volume mesh
    volume_mesh = mesh_from_label_volume(volume, spacing = im_scale)

    # get shape descriptors
    shape_descriptors.append({
        'surface_area': volume_mesh.area,
        'volume': volume_mesh.volume
    })
    
# go through shape descriptors and add to table
for i in shape_descriptors[0].keys():
    props_table[i] = [x[i] for x in shape_descriptors]
    
props_table

napari.manifest -> 'skan' could not be imported: Could not find file 'napari.yaml' in module 'skan'
100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 360/360 [00:02<00:00, 156.46it/s]


,label,mean_intensity,centroid-0,centroid-1,centroid-2,bbox-min-z,bbox-min-y,bbox-min-x,bbox-max-z,bbox-max-y,bbox-max-x,surface_area,volume
0,1,1.0,1.325000,8.000000,97.000000,0,6,95,4,11,100,662.743973,749.528747
1,2,1.0,1.788945,22.857621,117.403685,0,16,110,6,29,126,2403.845248,5758.004480
2,3,1.0,6.977041,14.812500,55.201531,0,3,49,14,23,61,4455.816428,7539.570347
3,4,1.0,2.991736,30.216942,69.904959,0,22,66,8,41,74,2899.116700,4647.242240
4,5,1.0,14.129329,43.701195,26.769844,0,31,17,31,56,39,12607.391094,31738.524160
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,356,1.0,125.542998,24.034398,48.024570,123,18,42,128,31,55,1785.647070,3903.453867
356,357,1.0,125.000000,63.000000,114.000000,123,61,112,128,66,117,797.279526,880.737280
357,358,1.0,125.299065,103.887850,103.616822,123,102,101,128,107,107,845.949527,1001.695147
358,359,1.0,125.000000,105.000000,6.000000,123,103,4,128,108,9,797.279526,880.737280
